# Project

Links:
- https://wandb.ai/andrewkho/wordle-solver/reports/Solving-Wordle-with-Reinforcement-Learning--VmlldzoxNTUzOTc4
- https://github.com/andrewkho/wordle-solver
- https://github.com/kgreed1/wordle-neural-net
- https://www.youtube.com/watch?v=v68zYyaEmEA (3b1b Information theory)

-> stupid idea. can be solved analytically

In [20]:
import random


MAX_ROUNDS = 6

State Vector of size 1+(26\*5\*3) = 391
 * [0] for the remaining turns
 * 5 \* 26 \* 3 for each letter in each position yes/maybe/no
     + [1, 0, 0] - char is definitely not in this spot
     + [0, 1, 0] - char is maybe in this spot
     + [0, 0, 1] - char is definitely in this spot

# Game

In [75]:
import wordle

In [78]:
vocab_subset_len = 100
random_seed = None
with open("data/possible_words.txt") as word_list:
    vocab = word_list.read().split('\n')
if vocab_subset_len:
    random.seed(random_seed)
    vocab = random.sample(vocab, k=vocab_subset_len)
random.seed(random_seed)
solution = random.choice(vocab)
game = Wordle(vocab, 6, solution)

In [79]:
game.play_visual()

sinus


silly


simon


sirme


siams


sigma


Game over, you  won!
Solution:  sigma


state space: 
2k actions you could take at each turn. 
After each of these steps, the game can respond with one of 3^5 combinations of Noes, Maybes, and Yeses. 
So you could think of this as (2k * 3^5)^6 possible sequences

# Model
### -> deep q learning

Reward: 
- +10 for each green 
- +5 for yellow
- (MAX_ROUNDS-n)+1*50 for win DOES NOT WORK!


Now: 
-50 for loose
+50 for win

# Agent